### 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

In [1]:
"""
90. 次単語予測
“The movie was full of”に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、
その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。
"""
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch.nn.functional as F

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
text = "The movie was full of"
encode_input = tokenizer(text, add_special_tokens=False, return_tensors="pt")
print("トークン列",tokenizer.convert_ids_to_tokens(encode_input["input_ids"][0]))



model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

with torch.no_grad():
    outputs = model(**encode_input)
    logits = outputs.logits

next_token_logits = logits[0, -1, :]
probs = F.softmax(next_token_logits, dim=-1)

top_k = 10
top_probs, top_indices = torch.topk(probs, top_k)

print("続くトークンの上位10語とその確率】")
for i in range(top_k):
    token = tokenizer.decode(top_indices[i].item())
    prob = top_probs[i].item()
    print(f"{i:2d}. {token:<15}  尤度: {prob:.6f}")


/home/hamamoto/100knocks_2025/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


トークン列 ['The', 'Ġmovie', 'Ġwas', 'Ġfull', 'Ġof']
続くトークンの上位10語とその確率】
 0.  jokes           尤度: 0.021892
 1.  great           尤度: 0.018644
 2.  laughs          尤度: 0.011524
 3.  bad             尤度: 0.010874
 4.  surprises       尤度: 0.010668
 5.  references      尤度: 0.010528
 6.  fun             尤度: 0.009992
 7.  humor           尤度: 0.007415
 8.  "               尤度: 0.007408
 9.  the             尤度: 0.006709


In [2]:
"""
91. 続きのテキストの予測
“The movie was full of”に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。
"""
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
text = "The movie was full of"
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

temperatures = [0.7, 1.0, 1.5]
num_texts = 3

for temp in temperatures:
    print(f"--- temperature {temp} ---")
    with torch.no_grad():
        input_ids = tokenizer.encode(
            text,
            add_special_tokens=False,
            return_tensors="pt"
        ).to(model.device)
        attention_mask = torch.ones(input_ids.shape, device=model.device)

        output_ids = model.generate(
            input_ids,
            max_length=20,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=temp,
            num_return_sequences=num_texts,
            pad_token_id=tokenizer.eos_token_id, 
            attention_mask=attention_mask
        )


        for i, output in enumerate(output_ids):
            generated_text = tokenizer.decode(output.tolist(), skip_special_tokens=True)
            print(f"テキスト {i+1}: {generated_text}\n")

--- temperature 0.7 ---
テキスト 1: The movie was full of drama and suspense, but its main characters were not.

"We

テキスト 2: The movie was full of twists and turns and a few memorable moments, including a sequence where a bunch

テキスト 3: The movie was full of great characters and memorable moments. The opening scene of "The Legend of Zelda

--- temperature 1.0 ---
テキスト 1: The movie was full of some truly weird and hilarious moments. We had to get into the character's

テキスト 2: The movie was full of funny moments that made me want to watch it.

Now, you

テキスト 3: The movie was full of the best stuff. It really brought the whole universe together with one of my

--- temperature 1.5 ---
テキスト 1: The movie was full of jokes. Not for nothing did a female-friendly guy mention how, at

テキスト 2: The movie was full of surprises! And as long as we had something great going (they always put

テキスト 3: The movie was full of twists: When the titular human doctor is trapped in the middle of Hell



In [3]:
"""
92. 予測されたテキストの確率を計算
“The movie was full of”に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。
"""
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

text = "The movie was full of"
max_new_tokens = 10

input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
print(f"{text} に続くテキスト")
with torch.no_grad():
    for _ in range(max_new_tokens):
        outputs = model(input_ids)
        logits = outputs.logits

        next_token_logits = logits[0, -1, :] #最後のトークンの次のトークンに対するすべてのスコア
        probs = F.softmax(next_token_logits, dim=-1) #確率に変換
        top_probs, top_indices = torch.topk(probs, 1)

        token = tokenizer.decode(top_indices[0].item())
        prob = top_probs[0].item()
        print(f"{token:<15}  尤度: {prob:.6f}")
        input_ids = torch.cat([input_ids,top_indices[0].unsqueeze(0).unsqueeze(0)], dim=1)

print(f"最終テキスト: {tokenizer.decode(input_ids[0])}")


The movie was full of に続くテキスト
 jokes           尤度: 0.021891
 and             尤度: 0.289224
 jokes           尤度: 0.098499
 about           尤度: 0.205555
 how             尤度: 0.099715
 the             尤度: 0.084637
 movie           尤度: 0.036412
 was             尤度: 0.296341
 a               尤度: 0.067677
 joke            尤度: 0.173508
最終テキスト: The movie was full of jokes and jokes about how the movie was a joke


In [4]:
"""
93. パープレキシティ
適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

The movie was full of surprises

The movies were full of surprises

The movie were full of surprises

The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。
"""

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

texts = ["The movie was full of surprises",
         "The movies were full of surprises",
         "The movie were full of surprises",
         "The movies was full of surprises"]
with torch.no_grad():
    for text in texts:
        input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
        outputs = model(input_ids, labels=input_ids)
        print(f"text: {text}, パープレキシティ: {torch.exp(outputs.loss).item()}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


text: The movie was full of surprises, パープレキシティ: 99.35449981689453
text: The movies were full of surprises, パープレキシティ: 126.48204040527344
text: The movie were full of surprises, パープレキシティ: 278.88226318359375
text: The movies was full of surprises, パープレキシティ: 274.6612854003906


In [5]:
"""
94. チャットテンプレート
“What do you call a sweet eaten after dinner?”という問いかけに対する応答を生成するため、
チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。
"""
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

question = "What do you call a sweet eaten after dinner?"
prompt =f"Question:{question},Answer:"

input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, do_sample=True,top_k=50,top_p=0.95, temperature=0.7,pad_token_id=tokenizer.eos_token_id)

response = tokenizer.decode(output[0], skip_special_tokens=True)

print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Question:What do you call a sweet eaten after dinner?,Answer:Serves 4-6.


In [6]:
"""
95. マルチターンのチャット
問題94で生成された応答に対して、追加で”Please give me the plural form of the word with its spelling in reverse order.”と問いかけたときの
応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。
"""
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

questions = ["What do you call a sweet eaten after dinner?",
             "Please give me the plural form of the word with its spelling in reverse order."]


with torch.no_grad():
    for question in questions:
        prompt =f"Question:{question},Answer:"
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
        output = model.generate(input_ids, max_length=60, num_return_sequences=1, do_sample=True,top_k=40,top_p=0.9, temperature=0.5,pad_token_id=tokenizer.eos_token_id)

        response = tokenizer.decode(output[0], skip_special_tokens=True)

        print(response)

Question:What do you call a sweet eaten after dinner?,Answer:I like to eat sweet foods. I think it is a good thing to eat a lot of sweet foods. I think it is a good thing to eat a lot of sweet foods. I think it is a good thing to eat
Question:Please give me the plural form of the word with its spelling in reverse order.,Answer:I am the plural form of the word with its spelling in reverse order.

Answer:Please give me the plural form of the word with its spelling in reverse order.

Answer:Please


In [19]:
"""
96. プロンプトによる感情分析
事前学習済み言語モデルで感情分析を行いたい。
テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、
SST-2の開発データにおける正解率を測定せよ。
"""
import pandas as pd
from tqdm import tqdm
import re

def create_prompt(text, template = ""):
    return f"Please classify the sentiment of the following texts as either Positive or Negative:{template}Question:{text},Answer:"

few_shot = """
Question: "hide new secretions from the parental units"
Answer: Negative
Question: "that loves its characters and communicates something rather beautiful about human nature"
Answer: Positive
Question: "contains no wit , only labored gags"
Answer: Negative
Question: "are more deeply thought through than in most ` right-thinking ' films."
Answer: Positive
"""
train_df = pd.read_csv("SST-2/train.tsv", sep="\t")
dev_df = pd.read_csv("SST-2/dev.tsv", sep="\t")

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

batch_size = 32

correct = 0
labels = []

for i in tqdm(range(0, len(dev_df), batch_size)):
    batch_df = dev_df.iloc[i:i+batch_size]
    batch_texts = batch_df["sentence"].tolist()
    prompts = [create_prompt(text, few_shot) for text in batch_texts]

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=256).to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=inputs["input_ids"].shape[1] + 10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for j, response in enumerate(responses):
        match = re.search(r"Answer:\s*(Positive|Negative)", response, re.IGNORECASE)
        if match:
            answer = match.group(1).lower()
            label = 1 if answer == "positive" else 0
        else:
            label = -1
        
        labels.append(label)
        if label == batch_df.iloc[j]["label"]:
            correct += 1

print(f"Acc : {correct / len(dev_df):.4f}")


100%|██████████| 28/28 [00:11<00:00,  2.51it/s]

Acc : 0.4908


In [ ]:
"""
97. 埋め込みに基づく感情分析
事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、
そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。
"""
import pandas as pd
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer
from torch.utils.data import DataLoader

train_df = pd.read_csv("SST-2/train.tsv", sep="\t")
dev_df = pd.read_csv("SST-2/dev.tsv", sep="\t")

class myDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = df["sentence"].tolist()
        self.labels = df["label"].tolist()
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=128
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

    def __len__(self):
        return len(self.labels)

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

train_dataset = myDataset(train_df, tokenizer)
dev_dataset = myDataset(dev_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=256, shuffle=True) 


In [21]:
import torch
from torch import nn
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from transformers import GPT2Tokenizer, GPT2Model

class CustomGPT2Classifier(nn.Module):
    def __init__(self, model_name="gpt2", num_labels=1):
        super().__init__()
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.gpt2 = GPT2Model.from_pretrained(model_name)
        self.classifier = nn.Linear(self.gpt2.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        last_token_indices = attention_mask.sum(dim=1) - 1
        pooled = hidden_states[torch.arange(hidden_states.size(0)), last_token_indices]
        logits = self.classifier(pooled)
        return logits
    
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].squeeze().float().to(device)

        pred = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = pred.squeeze()
        
        loss = loss_fn(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss


def evaluate(model, dev_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(dev_loader):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].squeeze().float().to(device)

            pred = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = pred.squeeze()
            preds = torch.sigmoid(logits) > 0.5

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    return acc


model = CustomGPT2Classifier()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

max_epochs = 3
for epoch in range(max_epochs):
    print(f"=================== {epoch+1} / {max_epochs} epoch ===================")
    
    avg_loss = train(model, train_loader, optimizer, loss_fn, device)
    print(f"Epoch {epoch+1}/{max_epochs}, Loss: {avg_loss:.4f}")

    acc = evaluate(model, dev_loader, device)
    print(f"Validation Accuracy: {acc:.4f}")

save_path = 'model/model97.pth' 
torch.save(model.state_dict(), save_path)

=================== 1 / 3 epoch ===================


100%|██████████| 2105/2105 [13:11<00:00,  2.66it/s]


Epoch 1/3, Loss: 0.2871


100%|██████████| 4/4 [00:04<00:00,  1.07s/it]


Validation Accuracy: 0.9106
=================== 2 / 3 epoch ===================


...

Validation Accuracy: 0.9083
=================== 3 / 3 epoch ===================


100%|██████████| 2105/2105 [13:45<00:00,  2.55it/s]


Epoch 3/3, Loss: 0.1323


100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


Validation Accuracy: 0.9197


In [22]:
"""
98. ファインチューニング
問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。
"""
import pandas as pd
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer
from torch.utils.data import DataLoader

def create_prompt(text, label = ""):
    return f"Please classify the sentiment of the following texts as either Positive or Negative:Question:{text},Answer:{label}"

train_df = pd.read_csv("SST-2/train.tsv", sep="\t")
dev_df = pd.read_csv("SST-2/dev.tsv", sep="\t")


class myGPT2Dataset(Dataset):
    def __init__(self, df, tokenizer):
        self.texts = df["sentence"].tolist()
        self.labels_num = df["label"].tolist()
        self.labels = ["Positive" if label == 1 else "Negative" for label in self.labels_num]
        self.prompts = [create_prompt(text, label) for text, label in zip(self.texts, self.labels)]
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokenized = self.tokenizer(
            self.prompts[idx],
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=128
        )
        input_ids = tokenized['input_ids'].squeeze(0)
        attention_mask = tokenized['attention_mask'].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': input_ids.clone()
        }

    def __len__(self):
        return len(self.labels)

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

train_dataset = myGPT2Dataset(train_df, tokenizer)
dev_dataset = myGPT2Dataset(dev_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=256, shuffle=True) 


In [ ]:
import torch
from torch import nn
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import GPT2LMHeadModel

def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].squeeze().long().to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss


def evaluate(model, dev_df, tokenizer, device, max_length=128):
    model.eval()
    correct = 0
    total = 0

    for _, row in tqdm(dev_df.iterrows(), total=len(dev_df), desc="Evaluating"):
        text = row['sentence']
        gold_label = "Positive" if row['label'] == 1 else "Negative"

        prompt = create_prompt(text)
        input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=max_length, truncation=True).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids,
                max_length=input_ids.shape[1] + 10,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=False,
                num_return_sequences=1,
            )

        decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        try:
            answer = decoded.split("Answer:")[-1].strip().split()[0]
        except:
            answer = ""

        if answer.lower() == gold_label.lower():
            correct += 1
        total += 1

    accuracy = correct / total
    return accuracy

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer)) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

max_epochs = 3
for epoch in range(max_epochs):
    print(f"=================== {epoch+1} / {max_epochs} epoch ===================")
    
    train_loss = train(model, train_loader, optimizer, loss_fn, device)
    print(f"Train Loss: {train_loss:.4f}")

    val_acc = evaluate(model, dev_df, tokenizer, device)
    print(f"Validation Accuracy: {val_acc * 100:.2f}%\n")

save_path = 'model/model98.pth' 
torch.save(model.state_dict(), save_path)


=================== 1 / 3 epoch ===================


Training: 100%|██████████| 2105/2105 [17:08<00:00,  2.05it/s]


Train Loss: 0.4507


Evaluating: 100%|██████████| 872/872 [00:38<00:00, 22.73it/s]


Validation Accuracy: 89.33%

=================== 2 / 3 epoch ===================


...

Validation Accuracy: 91.06%

=================== 3 / 3 epoch ===================


Training: 100%|██████████| 2105/2105 [17:11<00:00,  2.04it/s]


Train Loss: 0.3286


Evaluating: 100%|██████████| 872/872 [00:38<00:00, 22.38it/s]


Validation Accuracy: 91.51%



In [ ]:
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

load_path = 'model/model98.pth'
model.load_state_dict(torch.load(load_path, map_location=device))

model.eval()
with torch.no_grad():
    text = "this movie was amazing and made me cry"
    prompt = create_prompt(text)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=50, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)

Please classify the sentiment of the following texts as either Positive or Negative:Question:this movie was amazing and made me cry,Answer:Positive


In [26]:
"""
99. 選好チューニング
問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、
事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、
近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。
"""
import pandas as pd
train_df = pd.read_csv("SST-2/train.tsv", sep="\t")
dev_df = pd.read_csv("SST-2/dev.tsv", sep="\t")

from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

def preprocess_function(example):
    prompt = create_prompt(example["sentence"])
    label_text = "Positive" if example["label"] == 1 else "Negative"
    rejected_text = "Negative" if example["label"] == 1 else "Positive"

    return {
        "prompt": prompt,
        "chosen": label_text,
        "rejected": rejected_text
    }

train_dataset = train_dataset.map(preprocess_function)
dev_dataset = dev_dataset.map(preprocess_function)



Map: 100%|██████████| 872/872 [00:00<00:00, 9217.86 examples/s]


In [ ]:
from transformers import AutoModelForCausalLM
from trl import DPOConfig, DPOTrainer
import torch.distributed as dist
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)
config = DPOConfig(
    beta=0.1,
    learning_rate=5e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    save_steps=500,
    output_dir="model",
    ddp_find_unused_parameters=False,
    fsdp=[],
    fsdp_config={},
)

trainer = DPOTrainer(
    model,
    ref_model=None,
    args=config,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    processing_class=tokenizer
)
trainer.train()

In [39]:
model_dir = "model/checkpoint-3159"
tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
model = GPT2LMHeadModel.from_pretrained(model_dir)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

config = DPOConfig(
    beta=0.1,
    learning_rate=5e-6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    save_steps=500,
    output_dir="model",
    ddp_find_unused_parameters=False,
    fsdp=[],
    fsdp_config={},
)

trainer = DPOTrainer(
    model,
    ref_model=None,
    args=config,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    processing_class=tokenizer
)

eval_results = trainer.evaluate()
print(f"最終評価結果: {eval_results}")

Tokenizing eval dataset: 100%|██████████| 872/872 [00:02<00:00, 319.78 examples/s]


最終評価結果: {'eval_loss': 0.6931470632553101, 'eval_runtime': 14.971, 'eval_samples_per_second': 58.246, 'eval_steps_per_second': 0.935, 'eval_rewards/chosen': 1.9638468984339852e-07, 'eval_rewards/rejected': 5.5602651372055334e-08, 'eval_rewards/accuracies': 0.5122767686843872, 'eval_rewards/margins': 1.407820349186295e-07, 'eval_logps/chosen': -19.78765296936035, 'eval_logps/rejected': -61.800880432128906, 'eval_logits/chosen': -67.04113006591797, 'eval_logits/rejected': -62.113346099853516}


In [46]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import torch

def score(log_probs, input_ids, attention_mask):
    token_log_probs = torch.gather(log_probs, 2, input_ids.unsqueeze(-1)).squeeze(-1)
    token_log_probs = token_log_probs * attention_mask 
    return token_log_probs.sum(dim=1)

def pairwise_acc(model, tokenizer, eval_dataset):
    model.eval()
    dataloader = DataLoader(eval_dataset, batch_size=8)

    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            prompts = batch["prompt"]
            chosens = batch["chosen"]
            rejecteds = batch["rejected"]

            chosen_inputs = tokenizer([p + c for p, c in zip(prompts, chosens)],
                                      return_tensors="pt", padding=True, truncation=True)
            rejected_inputs = tokenizer([p + r for p, r in zip(prompts, rejecteds)],
                                        return_tensors="pt", padding=True, truncation=True)

            chosen_inputs = {k: v.to(model.device) for k, v in chosen_inputs.items()}
            rejected_inputs = {k: v.to(model.device) for k, v in rejected_inputs.items()}

            chosen_outputs = model(**chosen_inputs)
            rejected_outputs = model(**rejected_inputs)

            chosen_log_probs = torch.nn.functional.log_softmax(chosen_outputs.logits, dim=-1)
            rejected_log_probs = torch.nn.functional.log_softmax(rejected_outputs.logits, dim=-1)

            chosen_score = score(chosen_log_probs, chosen_inputs["input_ids"], chosen_inputs["attention_mask"])
            rejected_score = score(rejected_log_probs, rejected_inputs["input_ids"], rejected_inputs["attention_mask"])

            correct += (chosen_score > rejected_score).sum().item()
            total += len(prompts)

    return correct / total

dev_df = pd.read_csv("SST-2/dev.tsv", sep="\t")

model_dir = "model/checkpoint-3159"
tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
model = GPT2LMHeadModel.from_pretrained(model_dir)
model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

acc = pairwise_acc(model, tokenizer, dev_dataset)

print(f"Pairwise Accuracy: {acc * 100:.2f}%")


Pairwise Accuracy: 87.96%
